In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import tensorflow.keras.models as tkm
    
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(len(filenames), "-->", dirname)

In [8]:
image = mpimg.imread("/dataset/Training/meningioma_tumor/m (100).jpg")
plt.imshow(image, interpolation='nearest')
plt.title('Image plotting test', fontweight='bold')
plt.grid()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/dataset/Training/meningioma_tumor/m (100).jpg'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.,
                                   validation_split = 0.2)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
base_dir = "/kaggle/input/brain-tumor-classification-mri"
train_dir = "Training"
train_dir = os.path.join(base_dir, train_dir)

test_dir = "Testing"
test_dir = os.path.join(base_dir, test_dir)

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size = 20, 
                                                    class_mode = 'sparse', 
                                                    target_size = (224, 224), 
                                                    shuffle = True,
                                                    subset="training")

validation_generator = train_datagen.flow_from_directory(train_dir, 
                                                         batch_size = 20, 
                                                         class_mode = 'sparse', 
                                                         target_size = (224, 224), 
                                                         shuffle = True,
                                                         subset="validation")

In [ ]:
# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(test_dir,  
                                                  batch_size = 20, 
                                                  class_mode = 'sparse', 
                                                  target_size = (224, 224))

In [ ]:
from keras.applications.vgg16 import VGG16

vgg16 = VGG16(weights='imagenet', 
              input_shape=(224, 224, 3),
              #classes=self.classes, 
              include_top=False)

for layer in vgg16.layers:
            layer.trainable = False
        
vgg16.summary()

In [ ]:
x = layers.Flatten()(vgg16.output)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(4, activation='softmax')(x)

model = tkm.Model(vgg16.input, x)

model.summary()

In [ ]:
import tensorflow.keras.optimizers as tko

model.compile(optimizer = tko.RMSprop(learning_rate=0.0001), loss = 'crossentropy', metrics = ['acc'])
model.summary()

#vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)
vgghist = model.fit(train_generator, validation_data = validation_generator, epochs = 10)